<a href="https://colab.research.google.com/github/gorkham8/Story-Generator/blob/main/StoryGen.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import json

In [ ]:
!pip install PyPDF2 PyMuPDF transformers torch
from PyPDF2 import PdfReader
import fitz

In [ ]:
#turns pdf to json format for easier data manipulation
def pdf_to_json(pdf_file):
    pdf_data = []

    doc = fitz.open(pdf_file)

    current_object = {"title": "", "author": "", "content": ""}
    author_next_line = False
    extract_content = False

    for page_num in range(len(doc)):
        page = doc[page_num]

        #extract text from the page
        page_text = page.get_text()
        lines = page_text.split('\n')

        for i, line in enumerate(lines):
            if i == 1 or i == 2 or i == 3:
                if line.strip() == "By":  #check to identify line between title and author to get a reference point for each page
                    #start a new object if new title, author
                    if current_object["title"].strip() or current_object["author"].strip() or current_object["content"].strip():
                        pdf_data.append(current_object)
                    current_object = {"title": "", "author": "", "content": ""}

                    #extract title from line(s) above "By"
                    for j in range(i):
                        current_object["title"] += lines[j] + " "

                    #extract author from the line below "By"
                    author_line = lines[i + 1].strip() if i + 1 < len(lines) else ""
                    current_object["author"] = author_line

                    author_next_line = True
                    extract_content = False
            elif author_next_line:  #to avoid confusion after author stripped
                author_next_line = False
                extract_content = True
            elif extract_content & i != 0:
                current_object["content"] += line.strip() + " " #content extraction


    if current_object["title"].strip() or current_object["author"].strip() or current_object["content"].strip():
        pdf_data.append(current_object)

    return pdf_data


pdf_file = "drive/MyDrive/Colab/The Elder Scrolls.pdf"
json_data = pdf_to_json(pdf_file)

with open("drive/MyDrive/Colab/Denemeee.json", "w", encoding="utf-8") as json_file:
    json.dump(json_data, json_file, ensure_ascii=False, indent=4)


In [ ]:
with open("drive/MyDrive/Colab/The Elder Scroll Books.json", "r", encoding="utf-8") as json_file:
    json_data = json.load(json_file)

filtered_json_data = []

for obj in json_data:
    content = obj.get("content", "")  #getting content part of the JSON
    words = content.split()
    if len(words) >= 51: #cleaning data that is not long enough to train
        filtered_json_data.append(obj)

with open("drive/MyDrive/Colab/The Elder Scroll Books.json", "w", encoding="utf-8") as json_file:
    json.dump(filtered_json_data, json_file, ensure_ascii=False, indent=4)


In [ ]:
import nltk
nltk.download('popular')

[nltk_data] Downloading collection 'popular'
[nltk_data]    | 
[nltk_data]    | Downloading package cmudict to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/cmudict.zip.
[nltk_data]    | Downloading package gazetteers to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/gazetteers.zip.
[nltk_data]    | Downloading package genesis to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/genesis.zip.
[nltk_data]    | Downloading package gutenberg to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/gutenberg.zip.
[nltk_data]    | Downloading package inaugural to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/inaugural.zip.
[nltk_data]    | Downloading package movie_reviews to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping corpora/movie_reviews.zip.
[nltk_data]    | Downloading package names to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/names.zip.
[nltk_data]    | Downloading package shakespeare to /root/nltk_data...
[nlt

True

In [ ]:
from random import randint
from pickle import load
from keras.models import load_model
from keras.preprocessing.sequence import pad_sequences
import string
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
import json

nltk.download('wordnet')

with open("drive/MyDrive/Colab/The Elder Scroll Books.json", "r", encoding="utf-8") as json_file:
    json_data = json.load(json_file)


def load_content(index):
  return json_data[index]["content"]

def clean_text(txt):
  #to delete all square brackets and whatever is inside them
  pattern = r'\[.*]'
  txt = re.sub(pattern, '', txt)
  #to delete things that are absolutely unnecessary for use case in the project
  txt = txt.replace("\\s", " ")
  txt = txt.replace('..', '')
  txt = txt.replace("'s", "")
  txt = txt.replace("'d", "")
  txt = txt.replace("??", "?")
  txt = txt.replace("!!", "!")
  txt = txt.replace('...', '')
  txt = txt.replace('*', '')
  txt = txt.replace('#', '')

  tokens = txt.split()

  #lowercasing words and adding punctuations and numbers to tokens
  tokens = [word.lower() for word in tokens if word.isalpha() or word.isdigit() or word in ['.', ',', '?', '!']]

  return tokens


vocab = set()
sequences = list()
for i in range(300):
  i = len(json_data) - i - 1
  text = load_content(i)
  words = clean_text(text)
  print(text)
  print(words)
  print('Total Tokens: %d' % len(words))
  print('Unique Tokens: %d' % len(set(words)))
  vocab.update(words)
  print('Vocab Length: %d' % len(vocab))

  length = 51
  for i in range(length, len(words)):
    #sequence generation
    seq = words[i-length:i]
    # converting to line
    line = ' '.join(seq)

    sequences.append(line)
  print('Total Sequences: %d' % len(sequences))
  print()

with open("drive/MyDrive/Colab/TextGenDataset.txt", "w", encoding="utf-8") as text_file:
  for item in sequences:
    text_file.write("%s\n" % item)





[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


fellowship landed in long-boats upon the rocky shores of Hsaarik coast, but Ysgramor did not count his kin, Yngol’s among them. great gale darkened the sky. The seas thrashed and churned, and a the storm alone. carried him along the jagged coast. Two fortnights passed without long-boat was found in the icy surf, but the vengeful sea-ghosts had In his terrible grief, Ysgramor slew a dozen dozen beasts and burned Atmoran tradition, and Yngol was laid to rest with rites and honors first Children of the Sky to perish in Tamriel. 
['fellowship', 'landed', 'in', 'upon', 'the', 'rocky', 'shores', 'of', 'hsaarik', 'but', 'ysgramor', 'did', 'not', 'count', 'his', 'among', 'great', 'gale', 'darkened', 'the', 'the', 'seas', 'thrashed', 'and', 'and', 'a', 'the', 'storm', 'carried', 'him', 'along', 'the', 'jagged', 'two', 'fortnights', 'passed', 'without', 'was', 'found', 'in', 'the', 'icy', 'but', 'the', 'vengeful', 'had', 'in', 'his', 'terrible', 'ysgramor', 'slew', 'a', 'dozen', 'dozen', 'beasts

In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer
import numpy as np
import pickle

def encode_text(file_path):
  with open(file_path, 'r', encoding='utf-8') as file:
      text = file.read()

  sequence_length=51

  tokenizer = Tokenizer()
  tokenizer.fit_on_texts([text])
  sequences = tokenizer.texts_to_sequences([text])[0]

  tokenized_sequences = []
  for i in range(0, len(sequences), sequence_length):
      row = sequences[i:i+sequence_length]
      if len(row) < sequence_length:
          # pad the sequence if it's shorter than sequence_length
          row = row + [0] * (sequence_length - len(row))
      tokenized_sequences.append(row)

  tokenized_sequences = np.array(tokenized_sequences)
  vocab_size = len(tokenizer.word_index) + 1

  # saving the tokenizer for later use
  with open('drive/MyDrive/Colab/tokenizer300.pkl', 'wb') as fl:
    pickle.dump(tokenizer, fl)

  print("Tokenized sequences:", tokenized_sequences)
  print("Vocabulary size:", len(tokenizer.word_index) + 1)
  print(tokenized_sequences.shape)

  return tokenized_sequences, vocab_size


file_path = "drive/MyDrive/Colab/TextGenDataset.txt"
tokenized_sequences, vocab_size = encode_text(file_path)


Tokenized sequences: [[ 5616  3750     7 ...   177     1  3681]
 [ 2804     2 10356 ...  1015    82    28]
 [  983    15  5015 ...   482    55    91]
 ...
 [ 2845    19     1 ...     1 10069  1123]
 [    9     6   168 ...   168   477  1222]
 [   28   101    59 ...     0     0     0]]
Vocabulary size: 10712
(85448, 51)


In [ ]:
from tensorflow.keras.utils import to_categorical

# feature, target column splits by 50 to 1

X, y = tokenized_sequences[:,:-1], tokenized_sequences[:,-1]
y = to_categorical(y, num_classes=vocab_size)
seq_length = X.shape[1]

print(X)
print(X.shape)
print(y)
print(y.shape)

[[ 5616  3750     7 ...  9945   177     1]
 [ 2804     2 10356 ...    18  1015    82]
 [  983    15  5015 ...    14   482    55]
 ...
 [ 2845    19     1 ...    19     1 10069]
 [    9     6   168 ...     6   168   477]
 [   28   101    59 ...     0     0     0]]
(85448, 50)
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]]
(85448, 10712)


In [ ]:

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout
from tensorflow.keras.utils import plot_model

def define_model(vocab_size, seq_length):
  model = Sequential()
  model.add(Embedding(vocab_size, 50, input_length=seq_length))

  # bidirectional
  model.add(LSTM(100, return_sequences=True))

  model.add(LSTM(100))

  model.add(Dense(100, activation='relu'))
  model.add(Dense(vocab_size, activation='softmax'))

  model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

  model.summary()
  plot_model(model, to_file='model.png', show_shapes=True)
  return model

model = define_model(vocab_size, seq_length)


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 50, 50)            535600    
                                                                 
 lstm (LSTM)                 (None, 50, 100)           60400     
                                                                 
 lstm_1 (LSTM)               (None, 100)               80400     
                                                                 
 dense (Dense)               (None, 100)               10100     
                                                                 
 dense_1 (Dense)             (None, 10712)             1081912   
                                                                 
Total params: 1768412 (6.75 MB)
Trainable params: 1768412 (6.75 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


"some transformer model ideas\nfrom tensorflow.keras.models import Model\nfrom tensorflow.keras.layers import Input, Embedding, LSTM, Dense, Dot, Activation\nfrom tensorflow.keras.activations import softmax\nfrom tensorflow.keras import backend as K\n\ndef define_model(vocab_size, seq_length, units=100):\n    inputs = Input(shape=(seq_length,))\n\n    embedding = Embedding(vocab_size, 50, input_length=seq_length)(inputs)\n\n    lstm = LSTM(units, return_sequences=True)(embedding)\n\n    # Attention mechanism\n    attention = Dot(axes=[2, 2])([lstm, lstm])\n    attention = Activation('softmax')(attention)\n\n    # Applying attention weights\n    context = Dot(axes=[2, 1])([attention, lstm])\n    combined = K.concatenate([lstm, context])\n\n    output = Dense(combined, activation='relu')\n    output = Dense(vocab_size, activation='softmax')\n\n    model = Model(inputs=inputs, outputs=output)\n\n\n    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])\n

In [ ]:
from tensorflow.keras.callbacks import ReduceLROnPlateau

# ReduceLROnPlateau callback to avoid overfit
reduce_lr = ReduceLROnPlateau(monitor='accuracy', factor=0.1, patience=3, min_lr=0.00001)

model.fit(X, y, callbacks=[reduce_lr], batch_size=128, epochs=100)

model.save('drive/MyDrive/Colab/Lstm300.h5')


Epoch 1/100
668/668 [==============================] - 215s 315ms/step - loss: 7.0381 - accuracy: 0.0690 - lr: 0.0010
Epoch 2/100
668/668 [==============================] - 218s 326ms/step - loss: 6.5992 - accuracy: 0.0804 - lr: 0.0010
Epoch 3/100
668/668 [==============================] - 211s 315ms/step - loss: 6.3881 - accuracy: 0.0894 - lr: 0.0010
Epoch 4/100
668/668 [==============================] - 209s 313ms/step - loss: 6.1909 - accuracy: 0.1018 - lr: 0.0010
Epoch 5/100
668/668 [==============================] - 213s 318ms/step - loss: 6.0159 - accuracy: 0.1116 - lr: 0.0010
Epoch 6/100
668/668 [==============================] - 209s 313ms/step - loss: 5.8825 - accuracy: 0.1182 - lr: 0.0010
Epoch 7/100
668/668 [==============================] - 209s 314ms/step - loss: 5.7904 - accuracy: 0.1213 - lr: 0.0010
Epoch 8/100
668/668 [==============================] - 210s 314ms/step - loss: 5.6420 - accuracy: 0.1253 - lr: 0.0010
Epoch 9/100
668/668 [==============================] - 2

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [ ]:
from random import randint
import pickle
import numpy as np
from pickle import load
from keras.models import load_model
from keras.preprocessing.sequence import pad_sequences

def generate_text(model, tokenizer, input_text, max_length=100, temperature=1.0):
  input_sequence = tokenizer.texts_to_sequences([input_text])[0]

  generated_text = input_text
  for _ in range(max_length):
    padded_sequence = pad_sequences([input_sequence], maxlen=model.input_shape[1], padding='pre')

    probabilities = model.predict(padded_sequence)[0]

    next_word_index = sample_word_index(probabilities, temperature)

    next_word = tokenizer.index_word.get(next_word_index, '')

    generated_text += ' ' + next_word

    input_sequence.append(next_word_index)

  return generated_text

def sample_word_index(probabilities, temperature=0.2):
  # scaling to the probabilities
  scaled_probabilities = probabilities ** (1 / temperature)
  scaled_probabilities /= np.sum(scaled_probabilities)

  # Samples a word index
  word_index = np.random.choice(len(probabilities), p=scaled_probabilities)

  return word_index


model = load_model('drive/MyDrive/Colab/Lstm300.h5')

with open('drive/MyDrive/Colab/tokenizer300.pkl', 'rb') as f:
    tokenizer = pickle.load(f)

input_text = input("write input text here -> ")

generate_text(model, tokenizer, input_text, max_length=300, temperature=1.0)



write input text herethis is the history of
1/1 [==============================] - 0s 39ms/step


'this is the history of death pelagius was giving that indyk near had a time of morrowind with on the height of its armor took several orc in disguise and then his young shor and waited almost hard to make ruunvald a week off his temple king mantiarco before all many confidence that turned away from all she thinks he yet for the next was the memory of the eruption of duchess above me at response living flin on refused to the least i guild killed out daedra both storm swiftly a simple land before covered to waste a steps gems have asked me and all in their area in akavir the imperial family symmachus is quite occupied it s serious all without truly specific too prince hast had poured back down one i f emperor s history and other rock later in tamriel are in force tiber septim will boast with the sixth longer light and the blades but and on early earlier and then made effective brought water give unrest they located them in me before they are far the island do whiterun prince is nine lon